<a href="https://colab.research.google.com/github/Vaishnavig2909/Vaishnavig2909/blob/main/MI_MECH_RANDOM_FOREST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Random Forest Regression

In [ ]:
# hyperparameter tunned + learning rate model

In [26]:
#importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [57]:
#importing the dataset
dataset = pd.read_excel('/content/Data Ag_UPDATED _5_3_24.xlsx')
X = dataset.iloc[:, 0:6].values
y = dataset.iloc[:,6:9]

In [28]:
#print information from dataset
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Volume_Fraction      125 non-null    float64
 1   Flux                 125 non-null    float64
 2   Height               125 non-null    float64
 3   Total Time           125 non-null    float64
 4   Ambient_Temperature  125 non-null    float64
 5   Convection           125 non-null    float64
 6   Outlet_Temperatue    125 non-null    float64
 7   Thermal_Efficiency   125 non-null    float64
 8   Optical_Efficiency   125 non-null    float64
dtypes: float64(9)
memory usage: 8.9 KB


In [58]:
#splitting the data for training and testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 10)

In [59]:
y_test=np.asarray(y_test)

In [35]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
regressor.fit(X_train, y_train)

RandomForestRegressor(n_estimators=10, random_state=0)

In [36]:
# applying grid search for hyperparameter tuning
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [39]:
# Define the parameter grid for Random Forest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

In [40]:
grid_search= GridSearchCV(regressor, param_grid, cv=5, scoring='neg_mean_squared_error')

In [11]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=RandomForestRegressor(n_estimators=10, random_state=0),
             param_grid={'bootstrap': [True, False],
                         'max_depth': [None, 10, 20],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='neg_mean_squared_error')

In [12]:
# Print the best parameters found
best_params= grid_search.best_params_
print("Best Parameters:", best_params)

Best Parameters: {'bootstrap': True, 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


In [45]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
learning_rate=0.1
# Define the base estimators
base_estimators = [
    ('gb_rf', MultiOutputRegressor(GradientBoostingRegressor(n_estimators=100, learning_rate=learning_rate, random_state=42))),
    ('new_regressor', MultiOutputRegressor((RandomForestRegressor(**best_params))))
]

# Define the meta-estimator
meta_estimator = LinearRegression()

# Initialize the Stacking Regressor
stacked_model = StackingRegressor(estimators=base_estimators, final_estimator=meta_estimator)


In [60]:
print(y_train.shape) #just checking [93,3]

(93, 3)


In [61]:
# Fit the stacked model
stacked_model = MultiOutputRegressor(RandomForestRegressor())
stacked_model.fit(X_train, y_train)

MultiOutputRegressor(estimator=RandomForestRegressor())

In [62]:
y_pred=stacked_model.predict(X_test)

In [63]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score
# Initialize empty lists to store RMSE and R2 scores
rmse_scores = []
r2_scores = []

# Iterate over each column in y_test and y_pred
for i in range(y_test.shape[1]):
    # Calculate RMSE for the i-th column
    rmse = np.sqrt(mean_squared_error(y_test[:, i], y_pred[:, i]))
    rmse_scores.append(rmse)

    # Calculate R2 score for the i-th column
    r2 = r2_score(y_test[:, i], y_pred[:, i])
    r2_scores.append(r2)

# Print RMSE scores
print("Outlet_Temperatue Thermal_Efficiency  Optical_Efficiency")
for rmse in rmse_scores:
    print(rmse, end=' ')
print()

# Print R2 scores
for r2 in r2_scores:
    print(r2, end=' ')
print()


Outlet_Temperatue Thermal_Efficiency  Optical_Efficiency
9.254237316051858 3.9067775637143023 2.1053613178759627 
0.6831028534375803 0.9116014866831349 0.9523221644680593 
